In [1]:
import json
import pandas as pd
import os

In [2]:
course = 'EHS601'

with open(f'{course}/{course}-Quiz.json') as f:
    questions =json.load(f)
    
questions_df = pd.DataFrame(questions)
questions_df = questions_df[questions_df['correctAnswers']!= ""]
questions_df

,number,question,options,correctAnswers
0,1,Which three (3) government agencies regulate w...,"{'a': 'Public Health Agency of Canada', 'b': '...","a,b,c"
1,2,What can trigger a medical surveillance progra...,"{'a': 'Immune status of the individual', 'b': ...","a,b,c,d"
2,3,True or false? Risk group (RG) and containment...,"{'a': 'True', 'b': 'False'}",b
3,4,Which of the following are defined as biologic...,"{'a': 'Cell lines and cultures', 'b': 'Securit...","a,b,c,d"
5,6,Researchers at the University of Toronto are a...,{'a': 'An approved University of Toronto Biosa...,a
8,9,"At a minimum, what are three (3) examples of t...",{'a': 'A link to the U of T biosafety policies...,"a,b,c"
9,10,True or false? All laboratory rooms in which r...,"{'a': 'True', 'b': 'False'}",a
10,11,Who can handle biological materials at the Uni...,{'a': 'Anyone who has successfully completed t...,c
11,12,What needs to be listed on a biosafety permit?...,{'a': 'The individuals who will be working wit...,"a,b,c,d"
12,13,Sometimes you need to make changes to your Bio...,{'a': 'When you add a biological material or t...,"a,b"


## Defining Agent

In [4]:
from langchain.tools import BaseTool
from typing import Any, Dict, Optional

# This is just a dummy tool to avoid error for the agent
class DummyTool(BaseTool):
    name: str = "dummy_tool"  # Add type annotation here
    description: str = "This tool doesn't do anything and is only used to satisfy the requirement for at least one tool."
    
    def _run(self, query: str) -> str:
        return "Dummy tool was called but does nothing."
    
    async def _arun(self, query: str) -> str:
        return "Dummy tool was called but does nothing."

dummy_tool = DummyTool()  

In [ ]:
from dziner.agents import dZiner

tools = [dummy_tool]
tool_names = [tool.name for tool in tools]  
tool_desc = [tool.description for tool in tools]

PREFIX = """You are a helpful Chemist AI assistant called dZiner. You are tasked to answer a series of multiple choice questions relavant to lab safety.
    There could be more than one correct answer
    If you don't know, do not make up answers.
    Explain changes you make to the molecule in details at each step but do not put redundant information and make it short.
    """

FORMAT_INSTRUCTIONS = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:
'''
Thought: Here's your final answer:
Final Answer: [your response here]
'''

"""

SUFFIX = """Your final answer should be the letter "a", "d", or "a,b,c,d" if there are multiple correct answers. 
    Do not add anything else in the final answer

    Start by describing the problem: \n\nBegin!\n \n\nQuestion: {input}
Thought:{agent_scratchpad}\n"""



from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek
# llm_model = "gpt-4o"
llm_model = "DeepSeek-R1"#"o1-2024-12-17" #"claude-3-7-sonnet-20250219" #"claude-3-5-sonnet-20240620" #"claude-3-7-sonnet-20250219"

# agent_model = ChatAnthropic(model=llm_model, temperature=0.1, max_tokens=8192)

# agent_model = ChatOpenAI(
#                     # temperature=0.1,
#                     model_name=llm_model,
#                     request_timeout=1000,
#                     # max_tokens=4096,
#                 )

agent_model = ChatDeepSeek(
    model="...",
    temperature=0,
    max_tokens=None,
    timeout=8192)




agent = dZiner(tools, property="",
               model=agent_model, verbose=True, temp=0.1,get_cost=True,
              prefix=PREFIX, suffix=SUFFIX, format_instructions=FORMAT_INSTRUCTIONS).agent

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [7]:
from tqdm import tqdm
import time

for index, row in questions_df.iterrows():
    print("Question:", row['number'] ,"\n")
    question_prompt = row['question'] + '\n' + str(row['options'])
    # print(question_prompt)

    input_data = {
            "input": question_prompt ,
            "tools": tools,
            "tool_names": tool_names,
            "tool_desc": tool_desc
        }

    answer = agent.invoke(input_data)['output']
    questions_df.loc[index, 'inferredAnswers'] = answer.replace("'","").replace("\n", "") 
    # break
    # time.sleep(20)

Question: 1 



> Entering new AgentExecutor chain...


BadRequestError: Error code: 400 - {'error': {'message': 'Model Not Exist', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_request_error'}}

In [6]:
questions_df

,number,question,options,correctAnswers,inferredAnswers
0,1,Which three (3) government agencies regulate w...,"{'a': 'Public Health Agency of Canada', 'b': '...","a,b,c","a,b,c"
1,2,What can trigger a medical surveillance progra...,"{'a': 'Immune status of the individual', 'b': ...","a,b,c,d","a,b,d"
2,3,True or false? Risk group (RG) and containment...,"{'a': 'True', 'b': 'False'}",b,b
3,4,Which of the following are defined as biologic...,"{'a': 'Cell lines and cultures', 'b': 'Securit...","a,b,c,d","a,b,c,d"
5,6,Researchers at the University of Toronto are a...,{'a': 'An approved University of Toronto Biosa...,a,a
8,9,"At a minimum, what are three (3) examples of t...",{'a': 'A link to the U of T biosafety policies...,"a,b,c","a,b,c"
9,10,True or false? All laboratory rooms in which r...,"{'a': 'True', 'b': 'False'}",a,a
10,11,Who can handle biological materials at the Uni...,{'a': 'Anyone who has successfully completed t...,c,c
11,12,What needs to be listed on a biosafety permit?...,{'a': 'The individuals who will be working wit...,"a,b,c,d","a,b,c,d"
12,13,Sometimes you need to make changes to your Bio...,{'a': 'When you add a biological material or t...,"a,b","a,b,d"


In [7]:
import pandas as pd
import numpy as np

# Assuming questions_df already contains your data
# If not, you would load it from a CSV file like this:
# questions_df = pd.read_csv('your_data.csv')

def calculate_score(df):
    # Create a new column that checks if answers match exactly
    df['is_correct'] = df['inferredAnswers'] == df['correctAnswers']
    
    # Calculate the overall percentage score
    total_questions = len(df)
    correct_answers = df['is_correct'].sum()
    percentage_score = (correct_answers / total_questions) * 100
    
    # Create a detailed report
    results = {
        'total_questions': total_questions,
        'correct_answers': correct_answers,
        'percentage_score': percentage_score,
        'incorrect_questions': df[~df['is_correct']][['number', 'question', 'correctAnswers', 'inferredAnswers']]
    }
    
    return results

calculate_score(questions_df)

{'total_questions': 36,
 'correct_answers': 29,
 'percentage_score': 80.55555555555556,
 'incorrect_questions':     number                                           question correctAnswers  \
 1        2  What can trigger a medical surveillance progra...        a,b,c,d   
 12      13  Sometimes you need to make changes to your Bio...            a,b   
 15      16  How long does a lab need to keep equipment mai...              b   
 24      25  When removing cells from a liquid nitrogen tan...              b   
 29      30  Wen is using a piece of equipment that has rub...              a   
 34      35  You are creating a biosafety-related standard ...          a,b,c   
 35      36  Who do you contact for help with laboratory wa...              a   
 
    inferredAnswers  
 1            a,b,d  
 12           a,b,d  
 15               d  
 24             b,e  
 29             a,d  
 34         a,b,c,d  
 35             a,d  }

In [8]:
questions_df.to_csv(f'{course}_{llm_model}_3.csv', index=False, header=True)

## O1